In [1]:
# https://www.kaggle.com/hajekim/for-beginner/notebook
# https://www.kaggle.com/yongwookkimkor/samplesubmission11135
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns #시각화를 위한 라이브러리


## Bike Sharing Demand 진행방향
1. 훈련, 테스트 데이터셋의 형태 및 컬럼의 속성 데이터 값 파악
2. 데이터 전처리 및 시각화
3. 회귀모델 적용
4. 결론 도출

해당 데이터 세트에는 2011년 1월부터 2012년 12월까지 날짜/시간, 기온, 습도, 풍속 등의 정보를 기반으로 1시간 간격 동안의 자전거 대여 횟수가 기재돼 있습니다. 데이터 세트의 주요 칼럼은 다음과 같습니다. 이 중 결정 값은 맨 마지막 칼럼인 count로 '대여 횟수'를 의미합니다.

- datetime: hourly date + timestamp
- season: 1=봄, 2=여름, 3=가을, 4=겨울
- holiday: 1=주말을 제외한 국경일 등의 휴일, 0=휴일이 아닌 날
- workingday: 1=주말 및 휴일이 아닌 주중, 0=주말 및 휴일
- weather:
    - 1=맑음, 약간 구름 낀 흐림
    - 2=안개, 안개 + 흐림
    - 3=가벼운 눈, 가벼운 비 + 천둥
    - 4=심한 눈/비, 천둥/번개
- temp: 온도(섭씨)
- atemp: 체감온도(섭씨)
- humidity: 상대습도
- windspeed: 풍속
- casual: 사전에 등록되지 않은 사용자가 대여한 횟수
- registered: 사전에 등록된 사용자가 대여한 횟수
- count: 대여 횟수

In [2]:
#훈련데이터와 테스트 데이터 세트를 불러온다
train = pd.read_csv('../miniProject/input/train.csv', parse_dates=["datetime"])
test = pd.read_csv('../miniProject/input/test.csv', parse_dates=['datetime'])
print(train.shape)
print(test.shape)

(10886, 12)
(6493, 9)


In [3]:
#def ym(mydt):
#    return"{0}.{1}".format(mydt.year,mydt.month)

# train "datetime"로 year, month, day, hour, minute, second column을 추출한다.
train["year"] = train["datetime"].dt.year
#train["month"] = train["datetime"].dt.month
# V2.3에서 수정 month 항목 제외
train["day"] = train["datetime"].dt.day
train["hour"] = train["datetime"].dt.hour
# V2.0에서 수정
train["dayofweek"] = train["datetime"].dt.dayofweek
# V2.2에서 수정
#train['YM'] = train['datetime'].apply(ym)
#train['YM'] = train['datetime'].dt.year.astype('str') + train['datetime'].dt.month.astype('str')
train.shape

(10886, 16)

In [5]:
test["year"] = test["datetime"].dt.year
#test["month"] = test["datetime"].dt.month
# V2.3에서 수정 month 항목 제외
test["day"] = test["datetime"].dt.day
test["hour"] = test["datetime"].dt.hour
# V2.0에서 수정
test["dayofweek"] = test["datetime"].dt.dayofweek
# V2.2에서 수정
#test['YM'] = test['datetime'].apply(ym)
#test['YM'] = test['datetime'].dt.year.astype('str') + test['datetime'].dt.month.astype('str')
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,day,hour,dayofweek
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,20,0,3
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,20,1,3
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,20,2,3
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,20,3,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,20,4,3


## Feature Selection

In [6]:
inttocat = ['season', 'holiday', 'workingday', 'weather','year','day','hour'] #as type으로 category 형변환 
for i in inttocat:
    train[i] = train[i].astype("category")
    test[i] = test[i].astype("category")

drop_columns = ['datetime','casual','registered']
train.drop(drop_columns, axis=1,inplace=True)
    
inttocat
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   season      10886 non-null  category
 1   holiday     10886 non-null  category
 2   workingday  10886 non-null  category
 3   weather     10886 non-null  category
 4   temp        10886 non-null  float64 
 5   atemp       10886 non-null  float64 
 6   humidity    10886 non-null  int64   
 7   windspeed   10886 non-null  float64 
 8   count       10886 non-null  int64   
 9   year        10886 non-null  category
 10  day         10886 non-null  category
 11  hour        10886 non-null  category
 12  dayofweek   10886 non-null  int64   
dtypes: category(7), float64(3), int64(3)
memory usage: 587.0 KB


In [7]:
features = ["season", "weather", "temp", "atemp", "humidity", "windspeed",
                 'year', "hour", "dayofweek", "holiday", "workingday"]
features
# V2.0에서 수정
# "windspeed", "workingday" feature를 추가 0.48286 --> 0.41977로 줄어듬
# V2.3에서 수정
# "month" feature를 제외 0.48286 --> 0.41977로 줄어듬

['season',
 'weather',
 'temp',
 'atemp',
 'humidity',
 'windspeed',
 'year',
 'hour',
 'dayofweek',
 'holiday',
 'workingday']

In [8]:
X_train = train[features]

print(X_train.shape)
X_train.head(10)

(10886, 11)


,season,weather,temp,atemp,humidity,windspeed,year,hour,dayofweek,holiday,workingday
0,1,1,9.84,14.395,81,0.0000,2011,0,5,0,0
1,1,1,9.02,13.635,80,0.0000,2011,1,5,0,0
2,1,1,9.02,13.635,80,0.0000,2011,2,5,0,0
3,1,1,9.84,14.395,75,0.0000,2011,3,5,0,0
4,1,1,9.84,14.395,75,0.0000,2011,4,5,0,0
5,1,2,9.84,12.880,75,6.0032,2011,5,5,0,0
6,1,1,9.02,13.635,80,0.0000,2011,6,5,0,0
7,1,1,8.20,12.880,86,0.0000,2011,7,5,0,0
8,1,1,9.84,14.395,75,0.0000,2011,8,5,0,0
9,1,1,13.12,17.425,76,0.0000,2011,9,5,0,0


In [9]:
X_test = test[features]

print(X_test.shape)
X_test.head(10)

(6493, 11)


,season,weather,temp,atemp,humidity,windspeed,year,hour,dayofweek,holiday,workingday
0,1,1,10.66,11.365,56,26.0027,2011,0,3,0,1
1,1,1,10.66,13.635,56,0.0000,2011,1,3,0,1
2,1,1,10.66,13.635,56,0.0000,2011,2,3,0,1
3,1,1,10.66,12.880,56,11.0014,2011,3,3,0,1
4,1,1,10.66,12.880,56,11.0014,2011,4,3,0,1
5,1,1,9.84,11.365,60,15.0013,2011,5,3,0,1
6,1,1,9.02,10.605,60,15.0013,2011,6,3,0,1
7,1,1,9.02,10.605,55,15.0013,2011,7,3,0,1
8,1,1,9.02,10.605,55,19.0012,2011,8,3,0,1
9,1,2,9.84,11.365,52,15.0013,2011,9,3,0,1


In [10]:
label = 'count'
y_train = train[label]
y_train.head(10)

0    16
1    40
2    32
3    13
4     1
5     1
6     2
7     3
8     8
9    14
Name: count, dtype: int64

In [11]:
def rmsle(predicted_value, actual_value): #오류 값의 로그에 대한 RMSE
    predicted_value=np.array(predicted_value) # 예측값과 실제 값을 numpy 배열 형태로 변환
    actual_value=np.array(actual_value) # 예측값과 실제 값을 numpy 배열 형태로 변환
    
    log_predict=np.log(predicted_value+1) # 예측값과 실제 값에 1을 더하고 로그 변환
    log_actual=np.log(actual_value+1) # 예측값과 실제 값에 1을 더하고 로그 변환
    
    diff=log_predict-log_actual # 로그 변환한 예측값에서 로그 변환한 실제 값을 빼고 제곱
    diff=np.square(diff) # 로그 변환한 예측값에서 로그 변환한 실제 값을 빼고 제곱
    
    mean_diff=diff.mean() # 위에서 계산한 값의 평균
    
    score=np.sqrt(mean_diff) # 위에서 계산한 값에 루트
    return score

In [12]:
from sklearn.metrics import make_scorer 
rmsle_scorer=make_scorer(rmsle)
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from pandas import DataFrame
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

kfold=KFold(n_splits=10, shuffle=True, random_state=42) #K Fold 교차검증

In [12]:
#1 Random Forest 방식 사용


#model =RandomForestRegressor(n_estimators=100, n_jobs=-1,random_state=60)

#%time score=cross_val_score(model, X_train, y_train,cv=kfold, scoring=rmsle_scorer )
#print('Rmsle Random Forest Score:',score.mean())

Wall time: 10.6 s
Rmsle Random Forest Score: 0.3224684020954607


In [13]:
#HistGridentBoosting
# V2.3에서 HistGridentBoosting방법으로 
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

hgbm = HistGradientBoostingRegressor();

y_train_log = np.log1p(y_train)
hgbm.fit(X_train, y_train_log)

preds = hgbm.predict(X_train)

%time score=cross_val_score(hgbm, X_train, y_train,cv=kfold, scoring=rmsle_scorer )
print('Rmsle Random Forest Score:',score.mean())

#score = rmsle(np.exp(y_train_log), np.exp(preds), False)
#print("RMSLE Value For HistGradientBoostingRegressorr Regression : ", score)

C:\Users\difud\AppData\Local\Temp/ipykernel_19628/1008469715.py:6: RuntimeWarning: invalid value encountered in log
  log_actual=np.log(actual_value+1) # 예측값과 실제 값에 1을 더하고 로그 변환
C:\Users\difud\AppData\Local\Temp/ipykernel_19628/1008469715.py:6: RuntimeWarning: invalid value encountered in log
  log_actual=np.log(actual_value+1) # 예측값과 실제 값에 1을 더하고 로그 변환
C:\Users\difud\AppData\Local\Temp/ipykernel_19628/1008469715.py:6: RuntimeWarning: invalid value encountered in log
  log_actual=np.log(actual_value+1) # 예측값과 실제 값에 1을 더하고 로그 변환
C:\Users\difud\AppData\Local\Temp/ipykernel_19628/1008469715.py:6: RuntimeWarning: invalid value encountered in log
  log_actual=np.log(actual_value+1) # 예측값과 실제 값에 1을 더하고 로그 변환
C:\Users\difud\AppData\Local\Temp/ipykernel_19628/1008469715.py:6: RuntimeWarning: invalid value encountered in log
  log_actual=np.log(actual_value+1) # 예측값과 실제 값에 1을 더하고 로그 변환
C:\Users\difud\AppData\Local\Temp/ipykernel_19628/1008469715.py:6: RuntimeWarning: invalid value encountered i

Wall time: 8.29 s
Rmsle Random Forest Score: nan


C:\Users\difud\AppData\Local\Temp/ipykernel_19628/1008469715.py:6: RuntimeWarning: invalid value encountered in log
  log_actual=np.log(actual_value+1) # 예측값과 실제 값에 1을 더하고 로그 변환


In [14]:
# V2.0에서 수정
#y_train_log = np.log1p(y_train)

#model.fit(X_train, y_train_log)

NameError: name 'model' is not defined

In [14]:
#prediction=model.predict(X_test)
prediction=hgbm.predict(X_test)

prediction

array([2.42841342, 1.91295228, 1.38448943, ..., 4.67282998, 4.43506622,
       3.92451665])

In [15]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators=4000, alpha=0.01);

y_train_log = np.log1p(y_train)
gbm.fit(X_train, y_train_log)

preds = gbm.predict(X_train)

score = rmsle(np.exp(y_train_log), np.exp(preds))
print("RMSLE Gradient Boost:", score)

RMSLE Gradient Boost: 0.20417262895601523


In [16]:
df1 = pd.read_csv('../miniProject/input/sampleSubmission.csv')
# V2.0에서 수정
df1['count'] = np.expm1(prediction)
df1.to_csv('Score_{0:.5f}_test_submission.csv'.format(score), index=False)
df1.head(10)

# 0.39931 까지
# V2.2 버전에서 from sklearn.experimental import enable_hist_gradient_boosting from sklearn.ensemble import HistGradientBoostingRegressor 확인해보기

# V2.3에서 0.37534

,datetime,count
0,2011-01-20 00:00:00,10.340875
1,2011-01-20 01:00:00,5.773055
2,2011-01-20 02:00:00,2.992787
3,2011-01-20 03:00:00,2.045163
4,2011-01-20 04:00:00,1.964875
5,2011-01-20 05:00:00,4.979718
6,2011-01-20 06:00:00,32.534826
7,2011-01-20 07:00:00,108.119757
8,2011-01-20 08:00:00,231.940208
9,2011-01-20 09:00:00,136.715759
